
# Getting started

Once you've chosen your scenario, download the data from the Iowa website in csv format. Start by loading the data with pandas. You may need to parse the date columns appropriately.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sb
import seaborn.linearmodels as sblm
import numpy as np
import seaborn as sb
import seaborn.linearmodels as sblm
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn import metrics
from sklearn.linear_model import LinearRegression

def eda(dataframe):
    # This is the EDA function Ritika wrote and showed us. It is wonderful.
    print "Missing Values \n \n", dataframe.isnull().sum(),"\n"
    print "Duplicate Rows \n", dataframe.duplicated().sum(),"\n" # Added this
    print "Dataframe Types \n \n", dataframe.dtypes,"\n"
    print "Dataframe Shape \n", dataframe.shape,"\n"
    print "Dataframe Describe \n \n", dataframe.describe(include='all'),"\n"
    for feature in dataframe: 
        print feature
        print dataframe[feature].nunique()

In [ ]:
df=pd.read_csv('~/Downloads/Iowa_Liquor_sales_sample_10pct.csv')
df.columns=['date','store_number','city','zip','county_number','county','category','category_name','vendor_number','item_number','item_descript','bottle_ml','state_cost','state_retail','bottles_sold','sale','volume_sold_l','volume_sold_g']

In [ ]:
##Cleaning the data
df['state_cost'].replace('\$','',regex=True,inplace=True)
df['state_retail'].replace('\$','',regex=True,inplace=True)
df['sale'].replace('\$','',regex=True,inplace=True)

In [ ]:
##Chaniging column types
df[['state_cost','state_retail','sale']] = df[['state_cost','state_retail','sale']].apply(pd.to_numeric)
df[['store_number','vendor_number','item_number',]]=df[['store_number','vendor_number','item_number']].astype(object)
df['date']= pd.to_datetime(df['date'], format="%m/%d/%Y")

In [ ]:
##Adding in some columns that might be useful later on.
df['profit']=(df.state_retail-df.state_cost)*df.bottles_sold
df['ppb']=(df.state_retail-df.state_cost)
df['profit_margin']=(df.ppb/df.state_retail)*100

In [ ]:
##Replacing days of week numbers with day names
df['day_of_week'] = df['date'].dt.dayofweek
days = {0:'Mon',1:'Tues',2:'Wed',3:'Thurs',4:'Fri',5:'Sat',6:'Sun'}

In [ ]:
df['day_of_week'] = df['day_of_week'].apply(lambda x: days[x])

# Explore the data

Perform some exploratory statistical analysis and make some plots, such as histograms of transaction totals, bottles sold, etc.

In [ ]:
eda(df)

In [ ]:
##Just checking out the 425 dollar bottle, it looks ok
df[df.state_cost==425]
df[df.item_number==995381].item_descript

In [ ]:
#Looks like a typo 
df[df.bottles_sold==2508]

In [ ]:
df.drop(34867,inplace=True)

In [ ]:
df[(df['county_number'].isnull())&(df['county'].isnull())].head()

In [ ]:
##Replacing missing category number with name from the ILB website
df.loc[df.category ==1022200, 'category_name'] = 'TEQUILA'

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
##creating a function to look for broken zip codes
def zip_check(col):
    x=0
    for code in col:
        if str(code).isdigit() == False:
           x+=1 
    if x == 0:
        print "no bad zip codes"
    else:
        print "check "+str(x)+" zip codes"

In [ ]:
zip_check(df.zip)

In [ ]:
##712 is the area code for Dunlap, the proper zip code is 51529.
df.replace('712-2',51529,inplace=True)
df[df['zip']==51529].head()

In [ ]:
df['zip']=df['zip'].astype(int)

In [ ]:
df[df['zip']>53000]

In [ ]:
##The Iowa zip codes all begin with numbers between 50 and 52 anything that began with anything above that is 
##obviously a mistake. The real zipcode is 52601
df.zip.replace(56201,52601,inplace=True)

In [ ]:
#There doesn't seems to be any negative values for any columns were it would be a problem
print df[df['sale']<=0].shape
print df[df['bottle_ml']<0].shape
print df[df['state_cost']<=0].shape
print df[df['state_retail']<=0].shape
print df[df['bottles_sold']<=0].shape
print df[df['volume_sold_l']<=0].shape
print df[df['volume_sold_g']<=0].shape

In [ ]:
##Many columns are missing various values in category_name and county which are referenced elesewhere in the data-frame 

##Creating dictionaries of each category:category_name and county:county_number, and the opposite.
a=dict(zip(df.category.unique(),df.category_name.unique()))
b=dict(zip(df.category_name.unique(),df.category.unique()))
c=dict(zip(df.county_number.unique(),df.county.unique()))
d=dict(zip(df.county.unique(),df.county_number.unique()))

In [ ]:
#Referencing any category row that is True for our mask rule, and mapping over them using the dictionaries I created as a
#guide for the reference column.
mask1 = df.category_name.isnull()
mask2 = df.category.isnull()
mask3 = df.county.isnull()
mask4 = df.county_number.isnull()
df.loc[mask1,'category_name'] = df.loc[mask1, 'category'].map(a)
df.loc[mask2,'category'] = df.loc[mask2, 'category_name'].map(b)
df.loc[mask3,'county'] = df.loc[mask3, 'county_number'].map(c)
df.loc[mask4,'county_number'] = df.loc[mask4, 'county'].map(d)

In [ ]:
df.dropna(how='any',inplace=True)

In [ ]:
df.dtypes

In [ ]:
##I'm going to create a correlation heat map with all of my quantitative data to try to find  parameters to put into the model.
num_data=pd.DataFrame(df[['sale','volume_sold_l','bottles_sold','state_cost','state_retail','bottle_ml']],index=None)
num_data

In [ ]:
data_matrix = num_data.as_matrix()

In [ ]:
correlation_matrix = np.corrcoef(data_matrix.T)

In [ ]:
sb.heatmap(correlation_matrix,annot = True,linewidths = 0.5)

plt.show()

# Record your findings

Be sure to write out anything observations from your exploratory analysis.


In [ ]:
#There seems to be a quite obvious correlation between sale amount vs. volume sold in liters, 
#and sale amount vs. bottles sold. While it may seem like a good thing that we found such a strong paramter 
#for sale amount the clear lack of independence b/w all the variables, has a negative affect on our data that must be
#accounted for, either by dropping it completely or creating an interaction terms. There is a perfect correlation b/w state cost 
# and state retail, this is because the markup is an even 33% for all categories. 

# Mine the data

Now you are ready to compute the variables you will use for your regression from the data. For example, you may want to compute total sales per store from Jan to March of 2015, mean price per bottle, etc. Refer to the readme for more ideas appropriate to your scenario.

Pandas is your friend for this task. Take a look at the operations here for ideas on how to make the best use of pandas and feel free to search for blog and Stack Overflow posts to help you group data by certain variables and compute sums, means, etc. You may find it useful to create a new data frame to house this summary data.
 


In [ ]:
##Placing all of the sub-data-frames that I'm working with together to remove clutter
df15=pd.DataFrame(df[df['date'].dt.year==2015])
dfq115=pd.DataFrame(df15[(df15['date'].dt.year==2015)&(df15['date'].dt.month < 4)])
dfq23415=df15[df15['date'].dt.month > 3]
salesq115=dfq115['sale'].groupby(dfq115.store_number).sum()
salesq23415=dfq23415['sale'].groupby(dfq23415.store_number).sum()
sales15=df15['sale'].groupby(df15.store_number).sum()
salesfull=pd.DataFrame(sales15)
salesfull['quarter115_sales']=salesq115
dfq116=df[df['date'].dt.year==2016]
salesq116=dfq116['sale'].groupby(df.store_number).sum()
salesq116=pd.DataFrame(salesq116)
salesfull['quarter116_sales']=salesq116

In [ ]:
##Averages of basic stats for each store
df15stats=df15.groupby(['county','store_number']).agg({'state_retail':'mean','profit':'mean','ppb':'mean','volume_sold_l':'mean','bottles_sold':'mean','bottle_ml':'mean','sale':'sum','sale':'mean',})
df15stats  

In [ ]:
df.state_retail.mean()

In [ ]:
##this is a break down of each store contained in every city by the sum of sales for the first three months of 2015
dfq115.groupby(['county','store_number',dfq115['date'].dt.month]).agg({'sale':['sum']}).head()

In [ ]:
salesfull.dropna(how='any',inplace=True)

In [ ]:
##The first half of the week seems busier than the second probably because stores want to restock before the weekend
df15.groupby('day_of_week').agg({'sale':['sum']})

In [ ]:
top500=df15.groupby(df15.store_number).sum().nlargest(500,'sale').index

In [ ]:
##With some help from Roland 
topsales=df15[[store in top500 for store in df15.store_number]].groupby('store_number').agg(lambda x:x.value_counts().index[0])


In [ ]:
topsales

# Refine the data

Look for any statistical relationships, correlations, or other relevant properties of the dataset.

In [ ]:
# category_dummy=pd.get_dummies(topsales['day_of_week']

In [ ]:

# topsales_dummy['summed_sales']=df15.groupby('store_number').agg({'sale':'sum'}).nlargest(500,'sale')
# topsales_dummy.isnull().sum()
# X3 = topsales_dummy[]

# y3 = topsales_dummy['summed_sales']

# model1 = sm.OLS(y3, X3).fit() 
# predictions_dummy = model.predict(X3)
# model1.summary()

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
X = salesfull['quarter115_sales']
y = salesfull['sale']
X = sm.add_constant(X)
model = sm.OLS(y, X).fit() 
predictions15 = model.predict(X)
model.summary()

In [ ]:
model = sm.OLS(y, X).fit()
X2=salesfull['quarter116_sales']
X2=sm.add_constant(X2)
salesfull['quarter23416']=model.predict(X2)

In [ ]:
salesfull['full_2016']=salesfull['quarter23416']

In [ ]:
salesfull.head()
print salesfull.sale.sum()
print salesfull.full_2016.sum()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
linreg = LinearRegression()
linreg.fit(X_train, y_train)

In [ ]:
y_pred = linreg.predict(X_test)
print np.sqrt(metrics.mean_squared_error(y_test, y_pred))

In [ ]:
ridgereg = Ridge(alpha=.1, normalize=True)
ridgereg.fit(X_train, y_train)
y_pred = ridgereg.predict(X_test)
print np.sqrt(metrics.mean_squared_error(y_test, y_pred))

In [ ]:
plt.scatter(predictions15, y, s=30, c='b', marker='+', zorder=10)
plt.xlabel("Predicted Values from Q1")
plt.ylabel("Actual Values 2015")
plt.show()
print "MSE:", model.mse_model

In [ ]:
(275784905.11-273425332.18)/273425332.18*100

In [ ]:
## 2.3 million projected sales 